In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

from backtesting import Backtest, Strategy 
from backtesting.lib import crossover
from backtesting.test import SMA

In [4]:
msft = yf.Ticker("msft")

In [5]:
hist = msft.history(period="2y")

In [6]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-06-24,134.036829,135.406543,134.036829,134.799957,20628800,0.0,0
2019-06-25,134.281421,134.614064,129.859180,130.544037,33327400,0.0,0
2019-06-26,131.444178,132.804113,130.710400,131.033249,23657700,0.0,0
2019-06-27,131.238681,131.796359,130.622302,131.248459,16557500,0.0,0
2019-06-28,131.659398,131.688748,130.279892,131.062592,30043000,0.0,0
...,...,...,...,...,...,...,...
2021-06-17,256.070007,261.750000,256.010010,260.899994,27565500,0.0,0
2021-06-18,259.630005,262.299988,258.750000,259.429993,37090600,0.0,0
2021-06-21,259.820007,263.519989,257.920013,262.630005,26696100,0.0,0


In [7]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 30

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()

In [8]:
bt = Backtest(
    hist,
    SmaCross,
    cash=1000000,
    commission=0.005,
    margin=1.0,
    trade_on_close=True,
    exclusive_orders=True
)

output = bt.run()
print(output)

Start                     2019-06-24 00:00:00
End                       2021-06-23 00:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                    70.29703
Equity Final [$]               1397030.134744
Equity Peak [$]                1694419.114608
Return [%]                          39.703013
Buy & Hold Return [%]               97.262677
Return (Ann.) [%]                   18.156908
Volatility (Ann.) [%]               26.140001
Sharpe Ratio                         0.694602
Sortino Ratio                        1.183622
Calmar Ratio                         0.754395
Max. Drawdown [%]                  -24.068184
Avg. Drawdown [%]                   -3.930295
Max. Drawdown Duration      294 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                   11
Win Rate [%]                        45.454545
Best Trade [%]                       27.72858
Worst Trade [%]                     -9.005099
Avg. Trade [%]                    

In [10]:
bt.plot()
